# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,Fale old settlement,TK,-9.3852,-171.2468,29.02,75,46,4.88,1744681482
1,1,Saint Paul Harbor,US,57.7900,-152.4072,6.73,57,75,6.69,1744681352
2,2,Nhamundá,BR,-2.1861,-56.7131,23.74,93,75,0.96,1744681482
3,3,Klyuchi,RU,52.2667,79.1667,8.16,63,100,3.54,1744681482
4,4,Ålesund,NO,62.4723,6.1549,9.08,61,75,4.63,1744681483


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    scale=1,
    color="City",
    legend=True,
    alpha=0.6,
    frame_width=700,
    frame_height=500,
    hover_cols=["City", "Country", "Humidity"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions_df = city_data_df[
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_conditions_df = ideal_conditions_df.dropna()

# Display sample data
ideal_conditions_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
30,30,Inhambane,MZ,-23.8650,35.3833,24.22,85,0,3.44,1744681486
62,62,Galveston,US,29.3669,-94.9669,23.34,77,0,3.60,1744681552
151,151,Venice,US,27.0998,-82.4543,23.32,77,0,0.45,1744681478
202,202,Ghadames,LY,30.1337,9.5007,23.12,40,0,2.04,1744681752
263,263,Central,US,30.5544,-91.0368,22.01,79,0,4.12,1744681821


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_conditions_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
30,Inhambane,MZ,-23.8650,35.3833,85,
62,Galveston,US,29.3669,-94.9669,77,
151,Venice,US,27.0998,-82.4543,77,
202,Ghadames,LY,30.1337,9.5007,40,
263,Central,US,30.5544,-91.0368,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
Inhambane - nearest hotel: Hotel de Inhambane
Galveston - nearest hotel: Comfort Inn & Suites Texas City
Venice - nearest hotel: El Patio
Ghadames - nearest hotel: فندق عين الفارس
Central - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
30,Inhambane,MZ,-23.8650,35.3833,85,Hotel de Inhambane
62,Galveston,US,29.3669,-94.9669,77,Comfort Inn & Suites Texas City
151,Venice,US,27.0998,-82.4543,77,El Patio
202,Ghadames,LY,30.1337,9.5007,40,فندق عين الفارس
263,Central,US,30.5544,-91.0368,79,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    scale=1,
    color="orange",
    alpha=0.7,
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel Name", "City", "Country", "Humidity"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,City,Country)